# Inference with Dreambooth

We load pretrained models and ask to generate images based on the new vocabulary learned

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%ls

drive/  sample_data/


In [4]:
%cd drive/MyDrive/SketchToReality/

/content/drive/.shortcut-targets-by-id/1CWuL-Kv12lc1O0peSApcvTKUaHIxphWx/SketchToReality


In [5]:
%ls

Articles/                       dreambooth_cat_results/         results/
Data/                           dreambooth_cat_style_results/   results_dreambooth/
Dataset/                        dreambooth_parrot_results/      results_Llava_captioning/
data_viusalization_colab.ipynb  dreambooth_sequential_results/  textual_inversion_cat/
dog_dreambooth/                 hamburger_dreambooth/           utils/
dreambooth_animals/             helicopter_dreambooth/
dreambooth_cat/                 imageParrot/


In [ ]:
!pip install diffusers
!pip install accelerate
!accelerate config default
!pip install tqdm


# Loading the original images

We need this to get the labels corresponding to the images to generate captions for our dreambooth models

In [6]:
import pandas as pd
from PIL import Image
def dataframe_toPILlist(df, show_img = False):
    image_list = []
    label_list = []
    for i in range(df.shape[0]):
        row = df.iloc[i]
        path = row['path']
        lab = row['label']

        img = Image.open(path)
        image_list.append(img)
        label_list.append(lab)

        if show_img:
            print(f'\nLabel {lab}')
            display(img)

    return image_list, label_list

In [7]:
original_test_dataset_path = './Data/data/sketch/eval_image_captioning.csv'
eval_dataframe = pd.read_csv(original_test_dataset_path)
PILimages, labels = dataframe_toPILlist(eval_dataframe, show_img=False)

# Inference

In [34]:
from PIL import Image
import numpy as np
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0][0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img[0], box=(i%cols*w, i//cols*h))
    return grid

In [9]:
%ls

Articles/                       dreambooth_cat_results/         results/
Data/                           dreambooth_cat_style_results/   results_dreambooth/
Dataset/                        dreambooth_parrot_results/      results_Llava_captioning/
data_viusalization_colab.ipynb  dreambooth_sequential_results/  textual_inversion_cat/
dog_dreambooth/                 hamburger_dreambooth/           utils/
dreambooth_animals/             helicopter_dreambooth/
dreambooth_cat/                 imageParrot/


In [17]:
from torch import autocast
from diffusers import StableDiffusionPipeline
import torch

model_id = "./dreambooth_cat_results" #@param {type:"string"}
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [24]:
from tqdm import tqdm
def generate_images(pipe, labels, resample_if_nsfw=True):
    generated_images = []
    for l in tqdm(labels):
        prompt = f'A picture of {l} in the style of sks'
        num_samples=1
        image = pipe(prompt, num_images_per_prompt=num_samples, num_inference_steps=50, guidance_scale=7.5).images
        if resample_if_nsfw:
            n_try  =0
            while np.mean(image) == 0 and n_try<=100: # Totally black image, so mean of pixels is 0
                print('NSFW image generated... Trying resampling')
                image = pipe(prompt, num_images_per_prompt=num_samples, num_inference_steps=50, guidance_scale=7.5).images
                n_try+=1 # To be sure the algorithm stops, even if the image keeps being nsfw
        generated_images.append(image)

    return generated_images

In [25]:
generated_images_sks_cat = generate_images(pipe, labels)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  1%|          | 1/100 [00:08<13:45,  8.34s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 2/100 [00:16<13:51,  8.48s/it]

...

NSFW image generated... Trying resampling


  0%|          | 0/50 [00:00<?, ?it/s]

 99%|█████████▉| 99/100 [22:50<00:25, 25.67s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 100/100 [22:59<00:00, 13.79s/it]


In [33]:
for i, img in enumerate(generated_images_sks_cat): # Number will be correspond to the index in eval
  img[0].save(f'./results_dreambooth/images_sks_cat/img{i}.png')

In [38]:
grid = image_grid(generated_images_sks_cat, 10, 10)
grid

In [39]:
model_id = "./dreambooth_cat_style_results"
pipe= StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
generated_images_cat_style_sks = generate_images(pipe, labels)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  1%|          | 1/100 [00:08<13:39,  8.28s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 2/100 [00:16<13:48,  8.45s/it]

...

  0%|          | 0/50 [00:00<?, ?it/s]

 71%|███████   | 71/100 [12:21<04:00,  8.30s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

 72%|███████▏  | 72/100 [12:29<03:51,  8.25s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
for i, img in enumerate(generated_images_cat_style_sks): # Number will be correspond to the index in eval
  img[0].save(f'./results_dreambooth/image_cat_style_sks/img{i}.png')
grid2 = image_grid(generated_images_cat_style_sks, 10, 10)

In [ ]:
model_id = "./dreambooth_sequential_results"
pipe= StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

In [ ]:
generated_images_sequential = generate_images(pipe, labels)

In [ ]:
for i, img in enumerate(generated_images_sequential): # Number will be correspond to the index in eval
  img[0].save(f'./results_dreambooth/image_sequential/img{i}.png')
grid3 = image_grid(generated_images_cat_style_sks, 10, 10)